In [ ]:
import pandas as pd


def prepare_diary(path: str, timezone: str) -> tuple[pd.DataFrame, pd.Series]:
    df = pd.read_csv(path, on_bad_lines='skip')
    df.columns = df.columns.str.strip().str.lower()
    df['entry_type'] = df['entry_type'].str.strip().str.lower()

    day_types = df.loc[df['entry_type'] == 'day_type', ['date', 'type']].set_index('date')['type']
    df = df.loc[df['entry_type'] == 'period']
    df['type'] = df['type'].str.strip().str.lower()
    # df['context'] = df.apply(lambda row: day_types.get(row['date'], 'unknown'), axis=1)

    df['after_midnight'] = False
    df['start_time'] = pd.to_timedelta(df['start_time'] + ':00')
    df['end_time'] = pd.to_timedelta(df['end_time'] + ':00')
    df.loc[df['end_time'] < df['start_time'], 'after_midnight'] = True

    df['date'] = pd.to_datetime(df['date'])
    df['start_time'] = df['date'] + df['start_time']
    df['end_time'] = df['date'] + df['end_time']

    after_midnight = df[df['after_midnight']].index
    df.loc[after_midnight, 'start_time'] = df.loc[after_midnight, 'start_time'] - pd.Timedelta(days=1)

    df['start_time'] = df['start_time'].dt.tz_localize(timezone)
    df['end_time'] = df['end_time'].dt.tz_localize(timezone)

    df.rename(columns={'type': 'context', 'start_time': 'start', 'end_time': 'end'}, inplace=True)
    df = df[['context', 'start', 'end']]

    return [df, day_types]  # type: ignore

In [ ]:
import pandas as pd

from actimotus import Exposures

diary, days = prepare_diary('../temp/export_diary_100002.csv', 'Europe/Copenhagen')
sleep = diary.loc[diary['context'] == 'sleep', ['start', 'end']]

df = pd.read_parquet('../temp/export_100002_acc.parquet')

df['sleep'] = Exposures.context(
    df, sleep, 'sleep', ['lie', 'sit', 'kneel', 'squat']
)  # Thouhgh "kneel" and "squat" can't be detected as we use only thigh accelerometer
df['activity'] = df['activity'].cat.add_categories(['sleep'])
df.loc[df['sleep'], 'activity'] = 'sleep'

# # Calculating exposures
# exposures = Exposures(fused=False).compute(df)

df.index = df.index.tz_localize(None).tz_localize('Europe/Copenhagen')
Exposures().plot(df)

In [ ]:
import pandas as pd
from acti_motus import Exposures

df = pd.read_parquet('../temp/25CZ18.parquet')
df.sort_index(inplace=True)

Exposures().plot(df)